---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [114]:
! cp readonly/test.csv   test.csv

cp: cannot stat 'readonly/test.csv': No such file or directory


In [115]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('train.csv',encoding = "ISO-8859-1")
test_data = pd.read_csv('test.csv',encoding = "ISO-8859-1")
#train_data.dropna(inplace= True)


 

C:\Users\Lenovo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [116]:
train_data.drop(['payment_amount','payment_date','payment_status', 'balance_due', 'collection_status','compliance_detail'], axis = 1 , inplace= True)

In [117]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 28 columns):
ticket_id                     250306 non-null int64
agency_name                   250306 non-null object
inspector_name                250306 non-null object
violator_name                 250272 non-null object
violation_street_number       250306 non-null float64
violation_street_name         250306 non-null object
violation_zip_code            0 non-null float64
mailing_address_str_number    246704 non-null float64
mailing_address_str_name      250302 non-null object
city                          250306 non-null object
state                         250213 non-null object
zip_code                      250305 non-null object
non_us_str_code               3 non-null object
country                       250306 non-null object
ticket_issued_date            250306 non-null object
hearing_date                  237815 non-null object
violation_code                250306 non-null obj

## Feature Engineering

Checking for sparse columns as seen in info and dropping them if required 

In [118]:
train_data[train_data['grafitti_status'] == 'GRAFFITI TICKET']

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,non_us_str_code,country,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,compliance
250296,366178,"Buildings, Safety Engineering & Env Department","Bush, Wesley",MARK JACKSON,8020.0,PURITAN,NaN,251.0,HEYDEN,DETROT,MI,48219,NaN,USA,2006-07-28 14:00:00,2016-10-06 09:00:00,9-1-111,Failure of owner to remove graffiti or maintai...,Not responsible by City Dismissal,100.0,0.0,0.0,0.0,0.0,0.0,0.0,GRAFFITI TICKET,NaN


In [119]:
train_data['non_us_str_code'].nunique()

2

Dropping the above features along with other objects like Name and description

In [120]:
Id = test_data['ticket_id']


In [121]:
train_data.drop(['non_us_str_code','grafitti_status','violation_zip_code','violator_name','ticket_id','mailing_address_str_number'], axis=1, inplace= True)
test_data.drop(['non_us_str_code','grafitti_status','violation_zip_code','violator_name','ticket_id','mailing_address_str_number'], axis=1, inplace= True)

In [122]:
pd.set_option('display.max_columns', None)
train_data.head()

,agency_name,inspector_name,violation_street_number,violation_street_name,mailing_address_str_name,city,state,zip_code,country,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,compliance
0,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",2900.0,TYLER,S. WICKER,CHICAGO,IL,60606,USA,2004-03-16 11:40:00,2005-03-21 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0
1,"Buildings, Safety Engineering & Env Department","Williams, Darrin",4311.0,CENTRAL,Martin Luther King,Detroit,MI,48208,USA,2004-04-23 12:30:00,2005-05-06 13:30:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0,855.0,1.0
2,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",1449.0,LONGFELLOW,P.O. BOX,DETROIT,MI,48223,USA,2004-04-26 13:40:00,2005-03-29 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",1441.0,LONGFELLOW,ST. CLAIR,DETROIT,MI,48214,USA,2004-04-26 13:30:00,NaN,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",2449.0,CHURCHILL,CHURCHILL,DETROIT,MI,48206,USA,2004-04-26 13:00:00,2005-03-29 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [123]:
train_data['country'].value_counts()
train_data = train_data[train_data['country']== 'USA']
test_data = test_data[test_data['country']== 'USA']

In [124]:
train_data['ticket_issued_date'] = pd.to_datetime(train_data['ticket_issued_date']).dt.date
train_data['hearing_date'] = pd.to_datetime(train_data['hearing_date']).dt.date
test_data['ticket_issued_date'] = pd.to_datetime(test_data['ticket_issued_date']).dt.date
test_data['hearing_date']=pd.to_datetime(test_data['hearing_date']).dt.date
#train_data['Time_gap'] = train_data[train_data['hearing_date']- train_data['ticket_issued_date']]
train_data['ticket_issued_date']

0         2004-03-16
1         2004-04-23
2         2004-04-26
3         2004-04-26
4         2004-04-26
5         2004-05-01
6         2004-06-14
7         2004-06-16
8         2004-06-16
9         2004-07-12
10        2004-07-15
11        2004-07-16
12        2004-07-28
13        2004-08-05
14        2004-08-23
15        2004-08-23
16        2004-08-23
17        2004-09-07
18        2004-10-07
19        2004-10-07
20        2004-10-07
21        2004-10-13
22        2004-10-13
23        2004-10-28
24        2004-11-13
25        2004-11-13
26        2005-01-01
27        2005-01-13
28        2005-01-13
29        2005-01-13
             ...    
250276    2011-12-27
250277    2011-12-27
250278    2011-12-27
250279    2011-12-27
250280    2011-12-27
250281    2011-12-27
250282    2011-12-27
250283    2011-12-27
250284    2011-12-28
250285    2011-12-29
250286    2011-12-29
250287    2011-12-29
250288    2011-12-29
250289    2011-12-29
250290    2011-12-29
250291    2011-12-29
250292    201

In [125]:
train_data.dropna(subset= ['compliance'] ,inplace= True)


In [126]:
target =train_data['compliance']
train_data.drop(['compliance'], axis=1, inplace= True)

In [127]:
train_data.shape

(159869, 21)

In [128]:
target.shape

(159869,)

In [129]:
train_data.dtypes == 'object'

agency_name                  True
inspector_name               True
violation_street_number     False
violation_street_name        True
mailing_address_str_name     True
city                         True
state                        True
zip_code                     True
country                      True
ticket_issued_date           True
hearing_date                 True
violation_code               True
violation_description        True
disposition                  True
fine_amount                 False
admin_fee                   False
state_fee                   False
late_fee                    False
discount_amount             False
clean_up_cost               False
judgment_amount             False
dtype: bool

In [130]:
from category_encoders import *
enc = TargetEncoder(cols=['agency_name','inspector_name','violation_street_name','mailing_address_str_name','city',
'state','zip_code','country','ticket_issued_date', 'hearing_date', 'violation_code','violation_description','disposition'])
train_data = enc.fit_transform(train_data, target)
test_data = enc.transform(test_data)

In [131]:
train_data.shape

(159869, 21)

In [132]:
target.shape

(159869,)

In [133]:
test_data.shape

(61001, 21)

In [134]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159869 entries, 0 to 250293
Data columns (total 21 columns):
agency_name                 159869 non-null float64
inspector_name              159869 non-null float64
violation_street_number     159869 non-null float64
violation_street_name       159869 non-null float64
mailing_address_str_name    159869 non-null float64
city                        159869 non-null float64
state                       159869 non-null float64
zip_code                    159869 non-null float64
country                     159869 non-null float64
ticket_issued_date          159869 non-null float64
hearing_date                159869 non-null float64
violation_code              159869 non-null float64
violation_description       159869 non-null float64
disposition                 159869 non-null float64
fine_amount                 159869 non-null float64
admin_fee                   159869 non-null float64
state_fee                   159869 non-null float64
late_

In [135]:
import seaborn as sns
import matplotlib.pyplot as plt

In [148]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
def get_score(model):
    model = model()
    arr= (cross_val_score(model, train_data, target, cv=5 , n_jobs=-1, scoring= 'roc_auc'))
    print(arr.mean())
    return arr.mean()

In [137]:
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.ensemble import GradientBoostingClassifier as GB
c = get_score (GB)


<class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>


In [141]:
d = get_score(RF)

In [146]:
import xgboost as xgb
e = get_score(xgb.XGBClassifier)

C:\Users\Lenovo\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\Lenovo\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [149]:
e

0.9137333154523735

In [177]:
#model = xgb.XGBClassifier().pred
model = xgb.XGBClassifier().fit(train_data,target)
prediction =model.predict_proba(test_data)

In [212]:
prediction[1]

array([0.9938433, 0.0061567], dtype=float32)

In [237]:
prediction_1 = np.zeros(len(prediction))

In [238]:
for i in range(0,len(prediction)):
             prediction_1[i] = prediction[i][1]

In [239]:
result_1 = pd.Series(prediction_1,Id.values, name= 'compliance')

In [240]:
prediction_1.shape

(61001,)

In [242]:
result_1.to_csv('result_1')

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [244]:
b = pd.read_csv('result_1', names= ['ticket_id' , 'compliance'])
b.set_index('ticket_id',inplace= True)
b.head()

,compliance
ticket_id,
284932,0.056669
285362,0.006157
285361,0.055461
285338,0.000131
285346,0.315584


In [222]:
a = pd.read_excel('result_1.xlsx')

In [224]:
a.columns = ['ticket_id', 'compliance']

In [226]:
a.set_index('ticket_id', inplace= True)

In [230]:
a['compliance'].head()

ticket_id
284932    0.056669
285362    0.006157
285361    0.055461
285338    0.000131
285346    0.315584
Name: compliance, dtype: float64

In [231]:
def blight_model():
    import pandas as pd
    import numpy as np 
    a = pd.read_excel('result_1.xlsx')
    a.columns = ['ticket_id', 'compliance']
    a.set_index('ticket_id', inplace= True)
    return a['compliance']
    

In [ ]:
blight_model()